In [113]:
from PIL import Image
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import keras
import warnings
warnings.filterwarnings("ignore")

In [114]:
import nltk
from nltk.corpus import stopwords
import re
import joblib
from keras.preprocessing.text import Tokenizer
#import gensim
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ss\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [115]:
import keras

In [116]:
def cnn_model(path_loc):
    import tensorflow as tf
    import keras
    from keras.models import Sequential
    from keras.layers import Convolution2D
    from keras.layers import MaxPool2D
    from keras.layers import Flatten
    from keras.layers import Dense
    
    # Initializing CNN
        
    classifier = Sequential()
    
    # Step 1 : convolution
    classifier.add(Convolution2D(32,3,3,input_shape=(128,128,3),activation='relu'))

    # Step 2 : Pooling
    classifier.add(MaxPool2D(pool_size=(2,2)))

    # Adding second convolution layer.
    classifier.add(Convolution2D(32,3,3,activation='relu'))
    classifier.add(MaxPool2D(pool_size=(2,2)))
        
    # step 3 : flattening
    classifier.add(Flatten())

    # step 4 : Full connection
    classifier.add(Dense(units= 128,activation='relu')) 
    classifier.add(Dense(units=3,activation='softmax'))

    classifier.compile(optimizer='adam',metrics=['accuracy'],loss='categorical_crossentropy')

    # fitting CNN to the images

    from keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale = 1./255, # image agumentation
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
    test_datagen = ImageDataGenerator(rescale=1./255) # test data don't need much image agumentation.
    import os

    os.chdir(path_loc) # train and test path location
    training_set = train_datagen.flow_from_directory('F:/NIT/2020-New_projects/NITPIP03 - Social Image Sentiment Analysis/CODE/Original Code/Cnn_model/dataset',
                                                 target_size = (128, 128),
                                                 batch_size = 12,
                                                 class_mode = 'categorical') 

    test_set = test_datagen.flow_from_directory('F:/NIT/2020-New_projects/NITPIP03 - Social Image Sentiment Analysis/CODE/Original Code/Cnn_model/dataset',
                                            target_size = (128, 128),
                                            batch_size = 12,
                                           class_mode = 'categorical')

    # buliding model and performing model validation simultaneously
    history=classifier.fit_generator(training_set,
                         steps_per_epoch = 12, # number of images to be taken from training data set. 
                         epochs  = 15, 
                         validation_data = test_set,
                         validation_steps = 10) # number of images to be taken from test data set.

    return (training_set ,classifier,history)




In [117]:
training_set ,classifier,history= cnn_model('F:/NIT/2020-New_projects/NITPIP03 - Social Image Sentiment Analysis/CODE/Original Code/Cnn_model/dataset')

Found 638 images belonging to 3 classes.
Found 638 images belonging to 3 classes.
Epoch 1/15
12/12 [==============================] - 2s 207ms/step - loss: 0.7921 - accuracy: 0.5625 - val_loss: 0.6953 - val_accuracy: 0.5667
Epoch 2/15
12/12 [==============================] - 2s 138ms/step - loss: 0.7022 - accuracy: 0.5224 - val_loss: 0.6770 - val_accuracy: 0.5833
Epoch 3/15
12/12 [==============================] - 1s 112ms/step - loss: 0.6642 - accuracy: 0.6111 - val_loss: 0.6525 - val_accuracy: 0.5750
Epoch 4/15
12/12 [==============================] - 1s 107ms/step - loss: 0.6561 - accuracy: 0.6194 - val_loss: 0.6476 - val_accuracy: 0.7000
Epoch 5/15
12/12 [==============================] - 1s 100ms/step - loss: 0.6647 - accuracy: 0.6181 - val_loss: 0.6674 - val_accuracy: 0.5917
Epoch 6/15
12/12 [==============================] - 1s 100ms/step - loss: 0.7080 - accuracy: 0.5069 - val_loss: 0.6302 - val_accuracy: 0.6000
Epoch 7/15
12/12 [==============================] - 1s 90ms/step -

In [118]:
import numpy as np
from keras.preprocessing import image

In [119]:
%matplotlib inline
test_image = image.load_img('F:/NIT/2020-New_projects/NITPIP03 - Social Image Sentiment Analysis/CODE/Original Code/Cnn_model/dataset/Serving animals/9k_ (4).jpg', target_size = (128, 128))
test_image.show()

In [120]:
test_image = image.img_to_array(test_image)
test_image
np.shape(test_image) 

(128, 128, 3)

In [121]:
import inspect
function_name = inspect.stack()[0][3]
function_name

'<module>'

In [122]:
test_image = np.expand_dims(test_image, axis = 0)
test_image
np.shape(test_image)

(1, 128, 128, 3)

In [123]:
test = training_set.class_indices

In [124]:
result = classifier.predict_classes(test_image)
result 
result = result[0]
result

2

In [125]:
for key, value in test.items(): 
        if value == result: 
            print("The Predected image is", key)

The Predected image is Serving animals


In [126]:
train_df = pd.read_csv('F:/NIT/2020-New_projects/NITPIP03 - Social Image Sentiment Analysis/CODE/Original Code/Rnn_model/Rnn_code/training.1600000.processed.noemoticon.csv',encoding='latin-1',header=None)

In [127]:
train_df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [128]:
columns=['target','ids','date','flag','user','text']
train_df.columns=columns

In [129]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df.text)

In [130]:
stop_words=set(stopwords.words('english'))
stop_words.remove('not')

In [131]:
def preprocess(text):
    review=re.sub('@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+',' ',text)
    review=review.lower()
    review=review.split()
    review=[word for word in review if not word in stop_words]
    #print(review)
    review=pad_sequences(tokenizer.texts_to_sequences([review]), maxlen=300)
    return review

In [132]:
def text_prediction(review,text_model):
    review=preprocess(review)
    score=text_model.predict(review)
    score=score[0]
    if score<0.4:
        return "Negative"
    elif score>0.4 and score<0.6:
        return "Neutral"
    else:
        return "Positive"
    #print(score)

In [133]:
from keras.models import load_model

In [134]:
model_test = load_model('F:/NIT/2020-New_projects/NITPIP03 - Social Image Sentiment Analysis/CODE/Original Code/Cnn_model/sentiment_teja.h5')

In [135]:
def img_process(test_img,cnn_model):
    test_image = image.load_img(test_img, target_size = (128, 128))
    img_ = image.img_to_array(test_image)
    img_ = np.expand_dims(img_, axis = 0)
    #print(np.shape(img_))
    classes = training_set.class_indices
    result = cnn_model.predict_classes(img_)
    result = result[0]
    for key, value in classes.items(): 
        if value == result: 
            return "The Predected image is", key

In [136]:
def complete_predection(test_img,cnn_model,review,text_model):
    resp1 = img_process(test_img,cnn_model)
    #print(resp1)
    resp2 = text_prediction(review,text_model)
    #print(resp2)
    return resp1,resp2

In [137]:
text_model = load_model('F:/NIT/2020-New_projects/NITPIP03 - Social Image Sentiment Analysis/CODE/Original Code/Cnn_model/sentiment_teja.h5')

In [138]:
cnn_model = load_model('F:/NIT/2020-New_projects/NITPIP03 - Social Image Sentiment Analysis/CODE/Original Code/Cnn_model/cnn_model.h5')

In [139]:
test_img = "F:/NIT/2020-New_projects/NITPIP03 - Social Image Sentiment Analysis/CODE/Original Code/Cnn_model\dataset/Serving animals/9k_ (4).jpg"

In [140]:
review = "i love with my dog"

In [141]:
complete_predection(test_img,cnn_model,review,text_model)

(('The Predected image is', 'Serving animals'), 'Positive')

In [142]:
#resp1,resp2 = complete_predection(test_img,cnn_model,review,text_model)

In [143]:
#resp1

In [144]:
resp2

'Positive'